## 1) Текстовое описание набора данных

В качестве набора данных мы будем использовать набор данных 
по определению количественного показателя прогрессирования заболевания через год после исходного уровня

Датасет содержит следующие колонки:

- age - возраст в годах
- sex - пол
- bmi - индекс массы тела bmi
- bp - среднее артериальное давление bp
- s1 - tc, общий холестерин сыворотки
- s2 - ldl, липопротеины низкой плотности
- s3 - hdl, липопротеины высокой плотности
- s4 - tch, общий холестерин / ЛПВП
- s5 - ltg, логарифм уровня триглицеридов в сыворотке
- s6 - glu, уровень сахара в крови
- target - количественный показатель прогрессирования заболевания через год после исходного уровня

# Импорт библиотек
Импортируем библиотеки с помощью команды import. Как правило, все команды import размещают в первой ячейке ноутбука, но мы в этом примере будем подключать все библиотеки последовательно, по мере их использования.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")
from sklearn.datasets import *

: 

# Загрузка данных

Загрузим файлы датасета в помощью библиотеки sklearn.

Для дальнейшей работы необходимо преобразовать данные в DataFrame

In [ ]:
diabet = load_diabetes()
data=pd.DataFrame(data= np.c_[diabet['data'], diabet['target']],
                     columns= diabet['feature_names'] + ['target'])

: 

# 2) Основные характеристики датасета

In [ ]:
data.head()

: 

In [ ]:
# Размер датасета - 178 строк, 14 колонок
print('Всего строк: {}\nВсего колонок: {}'.format(data.shape[0], data.shape[1]))

: 

In [ ]:
data.dtypes

: 

In [ ]:
# Проверим наличие пустых значений
# Цикл по колонкам датасета
for col in data.columns:
    # Количество пустых значений - все значения заполнены
    temp_null_count = data[data[col].isnull()].shape[0]
    print('{} - {}'.format(col, temp_null_count))

: 

In [ ]:
data.describe()

: 

# 3) Визуальное исследование датасета
Для визуального исследования могут быть использованы различные виды диаграмм, мы построим только некоторые варианты диаграмм, которые используются достаточно часто.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.scatterplot(ax=ax, x='s2', y='s1', data=data, hue='target')

: 

## [Гистограмма](https://en.wikipedia.org/wiki/Histogram) 
Позволяет оценить плотность вероятности распределения данных.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.distplot(data['s2'])

: 

### Jointplot
Комбинация гистограмм и диаграмм рассеивания.

In [ ]:
sns.jointplot(x='s2', y='s1', data=data,  kind="hex")

: 

### "Парные диаграммы"
Комбинация гистограмм и диаграмм рассеивания для всего набора данных.

Выводится матрица графиков. На пересечении строки и столбца, которые соответстуют двум показателям, строится диаграмма рассеивания. В главной диагонали матрицы строятся гистограммы распределения соответствующих показателей.
С помощью параметра "hue" возможна группировка по значениям какого-либо признака.

In [ ]:
sns.pairplot(data, hue="target")

: 

### [Ящик с усами](https://en.wikipedia.org/wiki/Box_plot)
Отображает одномерное распределение вероятности.

In [ ]:
sns.boxplot(x='sex', y='target', data=data)

: 

### [Violin plot](https://en.wikipedia.org/wiki/Violin_plot)
Похоже на предыдущую диаграмму, но по краям отображаются распределения плотности

In [ ]:
sns.violinplot(x=data['target'])

: 

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,10))
sns.violinplot(ax=ax[0], x=data['target'])
sns.distplot(data['target'], ax=ax[1])

: 

Из приведенных графиков видно, что violinplot действительно показывает распределение плотности.

In [ ]:
sns.violinplot(x='sex', y='target', data=data)

: 

In [ ]:
sns.catplot(y='target', x='sex', data=data, kind="violin", split=True)

: 

# 4) Информация о корреляции признаков
Проверка корреляции признаков позволяет решить две задачи:
1. Понять какие признаки (колонки датасета) наиболее сильно коррелируют с целевым признаком (в нашем примере это колонка "target"). Именно эти признаки будут наиболее информативными для моделей машинного обучения. Признаки, которые слабо коррелируют с целевым признаком, можно попробовать исключить из построения модели, иногда это повышает качество модели. Нужно отметить, что некоторые алгоритмы машинного обучения автоматически определяют ценность того или иного признака для построения модели.
1. Понять какие нецелевые признаки линейно зависимы между собой. Линейно зависимые признаки, как правило, очень плохо влияют на качество моделей. Поэтому если несколько признаков линейно зависимы, то для построения модели из них выбирают какой-то один признак.

In [ ]:
data.corr()

: 

Корреляционная матрица содержит коэффициенты корреляции между всеми парами признаков.

Корреляционная матрица симметрична относительно главной диагонали. На главной диагонали расположены единицы (корреляция признака самого с собой).

На основе корреляционной матрицы можно сделать следующие выводы:
- Целевой признак наиболее сильно коррелирует с bmi (0.59) и ы5 - ltg, логарифмом уровня триглицеридов в сыворотке (0.57). Эти признаки обязательно следует оставить в модели.
- Целевой признак отчасти коррелирует с bp (0.44) и s4 - tch, общим холестерином / ЛПВП (0.43). Эти признаки стоит также оставить в модели.
- s1 - tc, общий холестерин сыворотки и s2 - ldl, липопротеины низкой плотности очень сильно коррелируют между собой (0.9). Поэтому из этих признаков в модели можно оставлять только один. 
- Также можно сделать вывод, что выбирая из s2 и s1 лучше выбрать s1, потому что он сильнее коррелирован с целевым признаком.

По умолчанию при построении матрицы используется коэффициент корреляции Пирсона. Возможно также построить корреляционную матрицу на основе коэффициентов корреляции Кендалла и Спирмена. На практике три метода редко дают значимые различия.

In [ ]:
fig, ax = plt.subplots(3, 1, sharex='col', sharey='row', figsize=(20,28))
sns.heatmap(data.corr(method='pearson'), ax=ax[0], annot=True, fmt='.2f')
sns.heatmap(data.corr(method='kendall'), ax=ax[1], annot=True, fmt='.2f')
sns.heatmap(data.corr(method='spearman'), ax=ax[2], annot=True, fmt='.2f')
fig.suptitle('Корреляционные матрицы, построенные различными методами')
ax[0].title.set_text('Pearson')
ax[1].title.set_text('Kendall')
ax[2].title.set_text('Spearman')

: 